In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import math

df2 = pd.read_csv(r"Y:\Krajina\GEOTUNDRA\Harmonized_4band_stack\indices_by_sensor_fin1_no_dupl.csv")
# 1) Read and prepare data

df2

In [ ]:
# map grid_code → ordered categorical class
class_map = {1: 'unvegetated', 2: 'dead grassland', 3: 'windswept grassland', 4: 'dense grassland'}
ordered = ['unvegetated', 'dead grassland', 'windswept grassland', 'dense grassland']
df1['class'] = pd.Categorical(
    df1['grid_code'].map(class_map),
    categories=ordered,
    ordered=True
)

# compute whisker percentiles for ±3σ
lower_pct = norm.cdf(-3) * 100
upper_pct = norm.cdf(3) * 100


def p_to_stars(p):
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    else:
        return 'ns'


def tukey_pvalues_for_pairs(df_long, pairs):
    res = pairwise_tukeyhsd(
        endog=df_long['value'],
        groups=df_long['group'],
        alpha=0.05
    )
    tbl = pd.DataFrame(res._results_table.data[1:], columns=res._results_table.data[0])
    tbl['group1'] = tbl['group1'].astype(int)
    tbl['group2'] = tbl['group2'].astype(int)

    lookup = {}
    for _, r in tbl.iterrows():
        g1, g2 = int(r['group1']), int(r['group2'])
        p = float(r['p-adj'])
        lookup[(g1, g2)] = p
        lookup[(g2, g1)] = p

    return {pair: lookup.get(pair, np.nan) for pair in pairs}


def add_pairwise_tukey_in_headroom(ax, col, headroom_frac=0.35):
    pairs = [(1, 2), (2, 3), (3, 4)]

    tmp = df1[['class', col]].dropna().copy()
    cls_to_num = {name: i for i, name in enumerate(ordered, start=1)}
    tmp['group'] = tmp['class'].astype(str).map(cls_to_num)
    tmp = tmp.rename(columns={col: 'value'})[['value', 'group']].dropna()

    if tmp['group'].nunique() < 2:
        return

    counts = tmp.groupby('group')['value'].size()
    if (counts < 2).any():
        return

    pvals = tukey_pvalues_for_pairs(tmp, pairs)

    y0, y1 = ax.get_ylim()
    y_range = y1 - y0
    y1_new = y1 + headroom_frac * y_range
    ax.set_ylim(y0, y1_new)

    headroom = y1_new - y1
    base = y1 + 0.22 * headroom
    step = 0.20 * headroom
    h = 0.06 * headroom
    star_gap = 0.005 * headroom

    for i, (g1, g2) in enumerate(pairs):
        p = pvals.get((g1, g2), np.nan)
        if not np.isfinite(p):
            continue

        y = base + i * step
        ax.plot([g1, g1, g2, g2], [y, y + h, y + h, y],
                lw=1.1, color='tab:blue')
        ax.text((g1 + g2) / 2, y + h + star_gap, p_to_stars(p),
                ha='center', va='bottom', fontsize=9, color='tab:blue')


def plot_box_with_annotations(ax, col):
    df1.boxplot(
        column=col,
        by='class',
        ax=ax,
        patch_artist=True,
        showfliers=False,
        whis=[lower_pct, upper_pct],
        return_type='dict',
        boxprops={'edgecolor': 'black', 'facecolor': 'white'},
        whiskerprops={'color': 'black', 'linewidth': 1},
        capprops={'color': 'black', 'linewidth': 1},
        medianprops={'color': 'darkorange', 'linewidth': 2},
        showmeans=False
    )

    ax.set_title(col, pad=10)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # aligned x ticks
    ax.set_xticks([1, 2, 3, 4])
    ax.set_xticklabels([1, 2, 3, 4])

    ax.tick_params(axis='both', which='both', length=0)
    for spine in ax.spines.values():
        spine.set_visible(True)
    ax.grid(False)

    offset_x = 0.22
    for i, cls in enumerate(ordered, start=1):
        data = df1.loc[df1['class'] == cls, col].dropna()
        if data.empty:
            continue

        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        iqr = q3 - q1
        y_offset = iqr * 0.03 if iqr != 0 else 1e-6

        ax.text(i + offset_x, q3 + y_offset, f'{q3:.2f}',
                va='bottom', ha='left', fontsize=7)
        ax.text(i + offset_x, q1 - y_offset, f'{q1:.2f}',
                va='top', ha='left', fontsize=7)

    add_pairwise_tukey_in_headroom(ax, col, headroom_frac=0.35)


# ---- figure layout: plot in the order listed ----
other_order = ['NBR', 'NDVI', 'NDWI', 'MSI1', 'MSI2', 'NDSVI', 'NDTI', 'STI', 'DFI']

# keep only columns that actually exist, preserving order
cols_to_plot = [c for c in other_order if c in df1.columns]
n = len(cols_to_plot)

# choose grid (3 columns like your original; adjust rows automatically)
ncols = 3
nrows = max(1, math.ceil(n / ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(12, 3.4 * nrows))
axes = np.atleast_1d(axes).ravel()

for i, col in enumerate(cols_to_plot):
    plot_box_with_annotations(axes[i], col)

# turn off unused axes (if any)
for j in range(n, len(axes)):
    axes[j].axis('off')

fig.suptitle('')
plt.tight_layout()
fig.savefig(r"U:\Paper_1\plot_Fig3.png", dpi=900, bbox_inches="tight")
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# -----------------------------
# 0) If df2 is not yet loaded, load it
# -----------------------------
# df2 = pd.read_csv("/mnt/data/indices_by_sensor_fin_no_dupl.csv")  # optional

# -----------------------------
# 1) Create class column (required)
# -----------------------------
class_map = {1: 'unvegetated', 2: 'dead grassland', 3: 'windswept grassland', 4: 'dense grassland'}
ordered = ['unvegetated', 'dead grassland', 'windswept grassland', 'dense grassland']

if "grid_code" not in df2.columns:
    raise KeyError("df2 must contain column 'grid_code' to build 'class'.")

df2 = df2.copy()
df2["class"] = pd.Categorical(
    df2["grid_code"].map(class_map),
    categories=ordered,
    ordered=True
)

# -----------------------------
# 2) Clean NoData (-9999 -> NaN) in index columns (recommended)
# -----------------------------
other_order = ['NBR', 'NDVI', 'NDWI', 'MSI1', 'MSI2', 'NDSVI', 'NDTI', 'STI', 'DFI']
cols = [c for c in other_order if c in df2.columns]

for c in cols:
    df2[c] = pd.to_numeric(df2[c], errors="coerce")
    df2[c] = df2[c].replace([-9999, -9999.0], np.nan)

# -----------------------------
# 3) Cohen's d (pooled SD) -> absolute value
# -----------------------------
def cohens_d_abs(x, y):
    x = np.asarray(pd.Series(x).dropna(), dtype=float)
    y = np.asarray(pd.Series(y).dropna(), dtype=float)
    nx, ny = len(x), len(y)
    if nx < 2 or ny < 2:
        return np.nan
    vx, vy = np.var(x, ddof=1), np.var(y, ddof=1)
    pooled_sd = np.sqrt(((nx - 1) * vx + (ny - 1) * vy) / (nx + ny - 2))
    if pooled_sd == 0 or not np.isfinite(pooled_sd):
        return np.nan
    return abs((np.mean(x) - np.mean(y)) / pooled_sd)

# -----------------------------
# 4) Class contrasts + table
# -----------------------------
pairs = [
    ('unvegetated', 'dead grassland', 'd_U_vs_D'),
    ('dead grassland', 'windswept grassland', 'd_D_vs_W'),
    ('windswept grassland', 'dense grassland', 'd_W_vs_De'),
]

rows = []
for col in cols:
    row = {'Index': col}

    for a, b, label in pairs:
        xa = df2.loc[df2['class'] == a, col]
        xb = df2.loc[df2['class'] == b, col]
        row[label] = cohens_d_abs(xa, xb)

    row['d_sum_all'] = np.nansum([row['d_U_vs_D'], row['d_D_vs_W'], row['d_W_vs_De']])
    row['d_sum_UD_DW'] = np.nansum([row['d_U_vs_D'], row['d_D_vs_W']])

    rows.append(row)

cohens_d_table = (
    pd.DataFrame(rows)
      .sort_values('d_D_vs_W', ascending=False)
      .reset_index(drop=True)
)

cohens_d_table
